<a href="https://colab.research.google.com/github/LeeSeungYun1020/HandsOnLargeLanguageModels/blob/main/handsOnLLM03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 대규모 언어 모델 개요

핸즈온 LLM 3장


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [4]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [85]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=False,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [86]:
generator = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    return_full_text = False,
    max_new_token = 128,
    do_sample = False,
)

Device set to use cuda
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [23]:
prompt = "Write an email apologizing to Yun for the tragic gardening mishap. Explain how it happened."
output = generator(prompt, do_sample = False, use_cache=False)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [26]:
print(output[0]["generated_text"])

 Express your regret and offer to help with the garden repairs.

Subject: Sincere Apologies for the Gardening Mishap

Dear Yun,

I hope this email finds you well. I am writing to express my deepest apologies for the unfortunate incident that occurred in your garden recently. It was truly a tragic mishap, and I am truly sorry for any distress or inconvenience it may have caused you.

The incident happened when I was attempting to help you with your gardening project. Unfortunately, due to my lack of experience and knowledge in this area, I unintentionally caused some damage to your plants. I understand that this was not my intention, and I deeply regret the outcome.

I realize that my actions have caused you a great deal of frustration and disappointment. Please know that I am truly sorry for any inconvenience this may have caused you. I take full responsibility for my actions and understand that it was my mistake.

To make amends, I would like to offer my assistance in repairing the da

In [27]:
print(model)

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out_features=3206

In [66]:
prompt = "The most famous fruit is the"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")
model_output = model.model(input_ids, use_cache=False)
lm_head_output = model.lm_head(model_output[0])

In [67]:
token_id = lm_head_output[0,-1].argmax(-1)
tokenizer.decode(token_id)

'apple'

In [68]:
for out in lm_head_output[0]:
  print(tokenizer.decode(out.argmax(-1)))

code
common
example
of
the
apple


In [69]:
model_output[0].shape

torch.Size([1, 6, 3072])

In [70]:
lm_head_output.shape

torch.Size([1, 6, 32064])

In [75]:
prompt = "The capital city of Korea is"
for i in range(1, 20):
  input_ids = tokenizer(prompt, return_tensors="pt").input_ids
  input_ids = input_ids.to("cuda")
  model_output = model.model(input_ids, use_cache=False)
  lm_head_output = model.lm_head(model_output[0])
  token_id = lm_head_output[0,-1].argmax(-1)
  print(tokenizer.decode(token_id))
  prompt = prompt + tokenizer.decode(token_id)

# Seoul은 두 단어로 나누어 토큰화되어 있음

Se
oul
.




##
#
Message
:


Trans
late
to
English
:




##
#


In [76]:
prompt = "Explain to me about the capital city of Korea."
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

In [87]:
%%timeit -n 1
out = model.generate(
    input_ids = input_ids,
    max_length = 128,
    use_cache = False
)
print(out)

tensor([[12027,  7420,   304,   592,  1048,   278,  7483,  4272,   310, 19109,
         29889,    13,    13,  1068, 13296,   918, 29871, 29896, 29901,  1068,
            13,    13,  1576,  7483,  4272,   310, 19109,   338,   922,  5059,
         29889,   922,  5059,   338,   278, 10150, 25311,   275,   297,  4275,
         19109,   322, 19700,   408,   278,  8604, 29892, 16375, 29892,   322,
         17407,  4818,   310,   278,  4234, 29889,   739,   338,  5982,   297,
           278,  6641,  5933,   760,   310,   278,  4234, 29892,  2978,   278,
          1261,   309,  3673,  1891, 10640,   313, 23560, 29999, 29897,   393,
          2903,  1078,   372,   515,  4644, 19109, 29889,    13,    13,  2008,
          5059,   338,  2998,   363,   967,  5400,  2071,   952, 29883,  2390,
           414, 29892,  1880, 29899, 11345,  1014,  1994, 29892,   322,  1835,
          9257, 29892,  3704,   476, 29899,  7323,   322, 22467,  8541,   294,
         29889,   450,  4272,   338,   884,  3271,  

In [88]:
%%timeit -n 1
out = model.generate(
    input_ids = input_ids,
    max_length = 128,
    use_cache = True
)
print(out)

tensor([[12027,  7420,   304,   592,  1048,   278,  7483,  4272,   310, 19109,
         29889,    13,    13,  1068, 13296,   918, 29871, 29896, 29901,  1068,
            13,    13,  1576,  7483,  4272,   310, 19109,   338,   922,  5059,
         29889,   922,  5059,   338,   278, 10150, 25311,   275,   297,  4275,
         19109,   322, 19700,   408,   278,  8604, 29892, 16375, 29892,   322,
         17407,  4818,   310,   278,  4234, 29889,   739,   338,  5982,   297,
           278,  6641,  5933,   760,   310,   278,  4234, 29892,  2978,   278,
          1261,   309,  3673,  1891, 10640,   313, 23560, 29999, 29897,   393,
          2903,  1078,   372,   515,  4644, 19109, 29889,    13,    13,    13,
          2008,  5059,   338,  2998,   363,   967,  5400,  2071,   952, 29883,
          2390,   414, 29892,  1880, 29899, 11345,  1014,  1994, 29892,   322,
          1835,  9257, 29892,  3704,   476, 29899,  7323,   322, 22467, 24615,
         29889,   450,  4272,   338,   884,  3271,  